# Population Modelling

<font size=3>The United States conducts a census every 10 years. The census has a breakdown of the population by state at the time during which it is conducted. This report attempts to use two models to predict the population of the different states. The two models are the exponential growth model and the logistical growth model</font>

In [25]:
#Imports of relevant libraries and initializing offline notebooks

#Graphing libraries
import plotly
import plotly.graph_objs as go

#Number libraries
import numpy as np
from scipy.optimize import curve_fit

#Set notebook mode to offline
plotly.offline.init_notebook_mode(connected=True)
# plotly.__version__

## Exponential Growth

There is no conception of a limited environment or constrained resources in a model of exponential population growth.

Exponential growth model of population makes the following assumptions about the populations changing:

* The rate of growth of the population is proportional to the size of the population.

This means that it only has to take only a few quantities into account

* $t = time$ (independent variable)
* $P = population$ (dependent variable)
* $k =  proportionality \ parameter$ (parameter) between rate of growth of the population and size of the population


$$
\begin{align}
\therefore \frac{dP}{dt} &= kP \\
\implies \frac{1}{P}\,dP &= k \,dt \\
\implies \int \frac{1}{P} \,dP &= \int k \,dt \\
\implies \ln P &= kt + c \\
\implies  P &= e^{kt + c} 
\end{align}
$$

$P = ae^{kt}+b$

The parameters $k$, $a$ and $c$ have to be determined so the function can be fitted to the data for predictive modelling. 

In [26]:
#defining the exponential growth function in code
def exponentialGrowth(t,k,a,b):
    return a*np.exp(k*t)+b

## Logistic Growth

Logistic growth ammends the exponential growth model to account for an environment with constained resources.

Logistic growth model of population makes the following assumptions about the populations changing:

* If the population is small, the rate of growth of the population is proportional to its size.
* If the population is too large to be supported by its environment and resources, the population will decrease.

The quantities it takes into account are:

* $t = time$ (independent variable)
* $P = population$ (dependent variable)
* $k =  proportionality \ parameter$ (parameter) between rate of growth of the population and size of the population
* $N = carrying \ capacity$  (parameter) 

$$
\begin{align}
\therefore \frac{dP}{dt} &= kP(1-\frac{P}{N}) \\
\implies k\,dt &= \frac{1}{P(1-\frac{P}{N})}\,dP\\
\implies \int k\,dt &= \int \frac{1}{P}\,dP + \int \frac{N}{1-\frac{P}{N}}\\
\implies kt + c &= \ln(P) - \ln(1-\frac{P}{N})\\
\implies  Ce^{kt} &= \frac{P}{1-\frac{P}{N}}\\
\implies  Ce^{kt} &= P + \frac{PCe^{kt}}{N}\\
\implies P &= (1+\frac{Ce^{kt}}{N})^{-1}(Ce^{kt})
\end{align}
$$

$P = (1+\frac{Ce^{kt}}{N})^{-1}(Ce^{kt}) + b$

The parameters $b$ , $C$, $k$ and $N$ have to be determined based on the values that fit the model best

In [86]:
def logisticGrowth(t,b,C,k,N):
    part1 = 1+((C*np.exp(k*t))/N)
    part2 = C*np.exp(k*t)
    
    return ((part2/part1)+b)
    

Both of these simple models are compared to the model for linear extrapolation. 

## Linear Extrapolation

Linear extrapolation makes the following assumptions about the population changing:

* rate of change of population is constant

The quantities is takes into account are:

* $t = time$ (independent variable)
* $P = population$ (dependent variable)
* $k =  proportionality \ parameter$ (parameter) between rate of growth of the population and size of the population

$$
\begin{align}
\therefore \frac{dP}{dt} &= k \\
\implies P &= kt + c
\end{align}
$$

In [87]:
#Linear extrapolation model
def linearExtrapolation(t,k,c):
    return (k*t) + c

## Population Data

The population data is obtained from the census from 1790 to 2000 and the modelling is done on this data in an attempt at predicting the population of the particular state in 2010

In [88]:
#Population Data Raw

populationData = {
    "masachusetts": {
        "years": np.array(range(1790,2010,10)),
        "population": np.array([379,423,472,523,610,738,995,1231,1457,1783,2239,2805,3366,3852,4250,4317,4691,5149,5689,5737,6016,6349])
    },
    "newYork": {
        "years": np.array(range(1790,2010,10)),
        "population": np.array([340,589,959,1373,1919,2429,3097,3881,4383,5083,6003,7269,9114,10385,12588,13479,14830,16782,18241,17558,17990,18976])
    },
    "northCarolina":{
        "years": np.array(range(1790,2010,10)),
        "population": np.array([394,478,556,639,738,753,869,993,1071,1400,1618,1893,2206,2559,3170,3572,4062,4556,5084,5880,6628,8049])
    }
}

# populationData

In [109]:
traceData = []
times = np.array(range(0, 22, 1))

timeFit = np.linspace(0,23,1000)
yearFit = np.linspace(1790,2020,1000)

for state in populationData.keys():
    
    trace = go.Scatter(x = populationData[state]["years"],
                       y = populationData[state]["population"],
                       mode = "markers",
                       name = state)
    
#   Exponential Fit
    popt, pcov = curve_fit(exponentialGrowth, times, populationData[state]["population"], p0=(1e-1, 1, 1))
    exponentialFit = exponentialGrowth(timeFit, *popt)
    
    traceExponentialFit = go.Scatter(x = yearFit,
                                     y = exponentialFit,
                                     mode = "lines",
                                     name = state + " exponential fit")
    
#   Logistic Fit
    popt, pcov = curve_fit(logisticGrowth, times, populationData[state]["population"], p0=(1, 1, 1e-5,100))
    logisticFit = logisticGrowth(timeFit, *popt)
    
    traceLogisticFit = go.Scatter(x = yearFit,
                                  y = logisticFit,
                                  mode = "lines",
                                  name = state + " logistic fit")
#   Linear Extrapolation
    popt = np.polyfit(times[-2:], populationData[state]["population"][-2:],1)
    print(popt)
    linearExtrapolate = linearExtrapolation(timeFit, *popt)
    
    traceLinearExtrapolate = go.Scatter(x = yearFit,
                                        y = linearExtrapolate,
                                        mode = "lines",
                                        name = state + " linear Extrapolate")
    
    traceData.append(trace)
    traceData.append(traceExponentialFit)
    traceData.append(traceLogisticFit)
    traceData.append(traceLinearExtrapolate)
    
layout = go.Layout(yaxis=dict(
                   rangemode='nonnegative',
                   autorange=True),
                   title="Population by State")

# layout = dict(title="Population by State")
fig = dict(data=traceData, layout=layout)

plotly.offline.iplot(fig, filename='dataPlot')

[ 333. -644.]
[  986. -1730.]
[  1421. -21792.]


In [22]:
trace0 = go.Scatter(x=np.asarray([1, 2, 3, 4]), 
                     y=np.asarray([4, 3, 2, 1]),
                     mode = 'lines+markers',
                     name = 'down')
trace1 = go.Scatter(x=np.asarray([1, 2, 3, 4]), 
                     y=np.asarray([1,2,3,4]),
                     mode = 'lines+markers',
                     name = 'up')

data = [trace0, trace1]

updatemenus = list([
    dict(active=-1,
         buttons=list([   
            dict(label = 'Up',
                 method = 'update',
                 args = [{'visible': [False, True]},
                         {'title': 'Up',
                          'annotations': []}]),
            dict(label = 'Down',
                 method = 'update',
                 args = [{'visible': [True, False]},
                         {'title': 'Down',
                          'annotations': []}]),
            dict(label = 'Both',
                 method = 'update',
                 args = [{'visible': [True, True]},
                         {'title': 'Both',
                          'annotations': []}])
        ]),
    )
])

layout = dict(title='Both', updatemenus=updatemenus)

fig = dict(data=data, layout=layout)

plotly.offline.iplot(fig, filename='basic-scatter')

In [ ]:
plotly.offline.iplot({
    "data": [go.Scatter(x=[1, 2, 3, 4], y=[4, 3, 2, 1])],
    "layout": go.Layout(title="hello world")
})

In [110]:
# Learn about API authentication here: https://plot.ly/python/getting-started
# Find your api_key here: https://plot.ly/settings/api

import plotly
import plotly.plotly as py
import plotly.graph_objs as go

# Scientific libraries
import numpy as np
from scipy.optimize import curve_fit


x = np.array([399.75, 989.25, 1578.75, 2168.25, 2757.75, 3347.25, 3936.75, 4526.25, 5115.75, 5705.25])
y = np.array([109,62,39,13,10,4,2,0,1,2])

def exponenial_func(x, a, b, c):
    return a*np.exp(-b*x)+c


popt, pcov = curve_fit(exponenial_func, x, y, p0=(1, 1e-6, 1))

xx = np.linspace(300, 6000, 1000)
yy = exponenial_func(xx, *popt)

# Creating the dataset, and generating the plot
trace1 = go.Scatter(
                  x=x,
                  y=y,
                  mode='markers',
                  marker=go.Marker(color='rgb(255, 127, 14)'),
                  name='Data'
                  )

trace2 = go.Scatter(
                  x=xx,
                  y=yy,
                  mode='lines',
                  marker=go.Marker(color='rgb(31, 119, 180)'),
                  name='Fit'
                  )

annotation = go.Annotation(
                  x=2000,
                  y=100,
                  text='$\textbf{Fit}: 163.56e^{-0.00097x} - 1.16$',
                  showarrow=False
                  )
layout = go.Layout(
                title='Exponential Fit in Python',
                plot_bgcolor='rgb(229, 229, 229)',
                  xaxis=go.XAxis(zerolinecolor='rgb(255,255,255)', gridcolor='rgb(255,255,255)'),
                  yaxis=go.YAxis(zerolinecolor='rgb(255,255,255)', gridcolor='rgb(255,255,255)'),
                  annotations=[annotation]
                )

data = [trace1, trace2]
fig = go.Figure(data=data, layout=layout)

plotly.offline.plot(fig, filename='Exponential-Fit-in-python')

/usr/lib/python3.6/site-packages/plotly/offline/offline.py:459: UserWarning:

Your filename `Exponential-Fit-in-python` didn't end with .html. Adding .html to the end of your file.



'file:///home/orgho/Documents/diffEQ/Exponential-Fit-in-python.html'